# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,54.93,75,100,17.83,SH,1725840584
1,1,madingou,-4.1536,13.5500,77.85,74,46,2.82,CG,1725840585
2,2,punta arenas,-53.1500,-70.9167,33.91,93,40,19.57,CL,1725840495
3,3,avarua,-21.2078,-159.7750,78.85,73,40,12.66,CK,1725840587
4,4,college,64.8569,-147.8028,54.07,82,100,4.61,US,1725840588


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_min = 65
ideal_temp_max = 75
ideal_humidity_min = 25
ideal_humidity_max = 55
ideal_cloudiness_max = 20
ideal_wind_speed_max = 5

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
ideal_city_data_df = city_data_df[
    (city_data_df["Max Temp"]>= ideal_temp_min)&
    (city_data_df["Max Temp"]<= ideal_temp_max)&
    (city_data_df["Humidity"]>= ideal_humidity_min)&
    (city_data_df["Humidity"]<= ideal_humidity_max)&
    (city_data_df["Cloudiness"]<= ideal_cloudiness_max)&
    (city_data_df["Wind Speed"]<= ideal_wind_speed_max)
]

ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,hamilton,39.1834,-84.5333,65.41,47,0,4.61,US,1725840494
287,287,macon,39.8667,-88.9668,66.25,37,0,3.44,US,1725840841
302,302,metropolis,37.1512,-88.7320,68.18,43,0,3.44,US,1725840936
346,346,gerdzhyush,37.5651,41.3916,74.57,36,0,3.62,TR,1725840994
393,393,dryden,49.7833,-92.7503,65.89,53,9,4.90,CA,1725841048
519,519,marivan,35.5219,46.1760,69.69,29,0,3.42,IR,1725841194
523,523,kenora,49.8167,-94.4337,72.57,49,20,4.61,CA,1725841199
548,548,decatur,39.8403,-88.9548,66.25,37,0,3.44,US,1725841042


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
63,hamilton,US,39.1834,-84.5333,47,
287,macon,US,39.8667,-88.9668,37,
302,metropolis,US,37.1512,-88.7320,43,
346,gerdzhyush,TR,37.5651,41.3916,36,
393,dryden,CA,49.7833,-92.7503,53,
519,marivan,IR,35.5219,46.1760,29,
523,kenora,CA,49.8167,-94.4337,49,
548,decatur,US,39.8403,-88.9548,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",  
    "apiKey": geoapify_key  
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hamilton - nearest hotel: North Vista Manor
macon - nearest hotel: Decatur Inn
metropolis - nearest hotel: Baymont Inn & Suites Metropolis
gerdzhyush - nearest hotel: Grand Lavin Otel
dryden - nearest hotel: Holiday Inn Express
marivan - nearest hotel: هتل میران
kenora - nearest hotel: Travelodge
decatur - nearest hotel: INTOWN MOTEL


,City,Country,Lat,Lng,Humidity,Hotel Name
63,hamilton,US,39.1834,-84.5333,47,North Vista Manor
287,macon,US,39.8667,-88.9668,37,Decatur Inn
302,metropolis,US,37.1512,-88.7320,43,Baymont Inn & Suites Metropolis
346,gerdzhyush,TR,37.5651,41.3916,36,Grand Lavin Otel
393,dryden,CA,49.7833,-92.7503,53,Holiday Inn Express
519,marivan,IR,35.5219,46.1760,29,هتل میران
523,kenora,CA,49.8167,-94.4337,49,Travelodge
548,decatur,US,39.8403,-88.9548,37,INTOWN MOTEL


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

%%capture --no-display


# Configure the map plot
hotel_map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600,
    hover_cols = ["City", "Country", "Hotel Name"]
)
# Display the map
hotel_map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)